## Set up dataset

In [ ]:
from robustness.model_utils import make_and_restore_model
from torch.hub import load_state_dict_from_url

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, RandomCrop, RandomHorizontalFlip, ToTensor, Normalize
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

from torchvision.models import resnet50

In [ ]:
ATTACK_EPS = 0.5
ATTACK_STEPSIZE = 0.1
ATTACK_STEPS = 10
NUM_WORKERS = 8
BATCH_SIZE = 10

In [ ]:
import torch as ch
from robustness.datasets import CIFAR
ds = CIFAR('/tmp')

In [ ]:
import tensorflow as tf
import torch as ch
from robustness.datasets import CIFAR
from robustness.tools.vis_tools import show_image_row
from robustness.attacker import AttackerModel

## Instantiate model

In [ ]:
from robustness.model_utils import make_and_restore_model
model, _ = make_and_restore_model(arch='resnet50', dataset=ds,
             resume_path='cifar_l2_0_5.pt')
model.eval()
pass


=> loading checkpoint 'C:/Users/32456/robust/newest/Cifar AT model/cifar_l2_0_5(Done).pt'
=> loaded checkpoint 'C:/Users/32456/robust/newest/Cifar AT model/cifar_l2_0_5(Done).pt' (epoch 153)


In [ ]:
# Load the saved model (attacker or illusionary model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attacker_model = models.resnet50(pretrained=False)
num_ftrs = attacker_model.fc.in_features
attacker_model.fc = nn.Linear(num_ftrs, 10)

loaded_attacker_model = models.resnet50(pretrained=False)
loaded_attacker_model.fc = nn.Linear(num_ftrs, 10)
loaded_attacker_model.load_state_dict(torch.load(r"resnet50_cifar10_model.pth"))
loaded_attacker_model = loaded_attacker_model.to(device)
state_dict = loaded_attacker_model.state_dict()


In [ ]:
# Set up dataset
ds = CIFAR('/tmp')

# Instantiate the defender model(robust model)
defender_model, _ = make_and_restore_model(arch='resnet50', dataset=ds,
             resume_path='cifar_l2_0_5.pt')
defender_model.eval()


# Set up loaders
NUM_WORKERS = 8
BATCH_SIZE = 10
_, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)

# Define the target representations
_, (im, label) = next(enumerate(test_loader))
im = im.to(device)  # Move the input data to the CUDA device
with ch.no_grad():
    (_, rep_defender), _ = defender_model(im, with_latent=True)

=> loading checkpoint 'C:/Users/32456/robust/newest/Cifar AT model/cifar_l2_0_5(Done).pt'
=> loaded checkpoint 'C:/Users/32456/robust/newest/Cifar AT model/cifar_l2_0_5(Done).pt' (epoch 153)
==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
    _, rep_defender = defender_model(inputs, with_latent=True, fake_relu=True)

In [ ]:
state_dict1 = loaded_attacker_model.state_dict()
rep_attacker = state_dict1['fc.weight']

In [ ]:
train_loader1, test_loader1 = ds.make_loaders(workers=0, batch_size=50000)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
_, (im_inv, label_inv) = next(enumerate(train_loader1)) # Images to invert

In [ ]:
df = numpy.save("rob.lI0.5", label_inv.cpu().numpy())

In [ ]:
df = numpy.save("im_invlI0.5", im_inv.cpu().numpy())

In [ ]:
im_inv = np.load('im_invlI0.5.npy')
im_inv=ch.from_numpy(im_inv)

In [ ]:
im_seed = ch.clamp(ch.randn_like(im_inv) / 20 + 0.5, 0, 1)

In [ ]:
robustified_images = ch.zeros((ds.shape[0]))
for t in range(5*1000):
    with ch.no_grad():

        (_, rep_inv), _ = model(im_inv[i:i+10].to(device), with_latent=True)  # Corresponding representation
       _ , im_matched = model(im_seed[i:i+10].to(device), rep_inv.to(device), make_adv=True, **kwargs)
    robustified_images[i*10+t] = im_matched
    i=i+10

In [ ]:
df = numpy.save("1roblI0.5", im_matched0.cpu().numpy())

In [ ]:
i

In [ ]:

# Set up dataset
ds = CIFAR('/tmp')

# Instantiate the defender model
defender_model, _ = make_and_restore_model(arch='resnet50', dataset=ds,
             resume_path='cifar_l2_0_5.pt')
defender_model.eval()


# Set up loaders
NUM_WORKERS = 8
BATCH_SIZE = 10
_, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)

# Define the target representations
_, (im, label) = next(enumerate(test_loader))
im = im.to(device)  # Move the input data to the CUDA device
with ch.no_grad():
    (_, rep_defender), _ = defender_model(im[0:10], with_latent=True)
rep_attacker=state_dict['fc.weight']

# Define the adversarial attack
def custom_loss(defender_model, inputs, targ):
    # Compute representations for the input
    _, rep_defender = defender_model.to(device)(inputs, with_latent=True, fake_relu=True)
    state_dict1 = loaded_attacker_model.to(device).state_dict()
    rep_attacker = state_dict1['fc.weight']
    # Compute losses for the defender and attacker models
    loss_defender = ch.div(ch.norm(rep_defender - targ, dim=1), ch.norm(targ, dim=1))
    loss_attacker = -ch.div(ch.norm(rep_attacker - targ, dim=1), ch.norm(targ, dim=1))

    # Compute weighted sum of losses
    loss = loss_defender + loss_attacker

    return loss, None

## Set up loaders

In [ ]:
kwargs = {
    'custom_loss': custom_loss,
    'constraint':'2',
    'eps': 1000,
    'step_size': 1,
    'iterations': 1000,
    'targeted': True,
    'do_tqdm': True,
}

In [ ]:
train_loader1, test_loader1 = ds.make_loaders(workers=0, batch_size=50000)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
_, (im_inv, label_inv) = next(enumerate(train_loader1)) # Images to invert

In [ ]:
im_seed = ch.clamp(ch.randn_like(im_inv) / 20 + 0.5, 0, 1)

In [ ]:
with ch.no_grad():
    (_, rep_inv0), _ = model(im_inv[0:10].to(device), with_latent=True)  # Corresponding representation

In [ ]:
robustified_images = ch.zeros((ds.shape[0]))
for t in range(5*1000):
    with ch.no_grad():

        (_, rep_inv), _ = model(im_inv[i:i+10].to(device), with_latent=True)  # Corresponding representation
       _ , im_matched = model(im_seed[i:i+10].to(device), rep_inv.to(device), make_adv=True, **kwargs)
    robustified_images[i*10+t] = im_matched
    i=i+10

Current loss: 0.1057443618774414: 100%|██████████| 1000/1000 [01:29<00:00, 11.22it/s]


Current loss: 0.10629600286483765: 100%|██████████| 1000/1000 [01:29<00:00, 11.19it/s]


Current loss: 0.07803093641996384: 100%|██████████| 1000/1000 [02:11<00:00,  7.60it/s]


Current loss: 0.10593223571777344: 100%|██████████| 1000/1000 [01:29<00:00, 11.18it/s]


Current loss: 0.10199970006942749: 100%|██████████| 1000/1000 [01:29<00:00, 11.16it/s]


...

Current loss: 0.09640828520059586: 100%|██████████| 1000/1000 [01:29<00:00, 11.12it/s]


Current loss: 0.09931565076112747: 100%|██████████| 1000/1000 [01:42<00:00,  9.76it/s]


Current loss: 0.09371892362833023: 100%|██████████| 1000/1000 [01:40<00:00,  9.98it/s]


Current loss: 0.1022213026881218: 100%|██████████| 1000/1000 [01:35<00:00, 10.46it/s]


Current loss: 0.09846687316894531: 100%|██████████| 1000/1000 [01:30<00:00, 11.08it/s]


In [ ]:
df = numpy.save("rob", im_matched0.cpu().numpy())
loaded_array = np.load('rob.npy')
loaded_array.shape

(50000, 3, 32, 32)

In [ ]:
df = numpy.save("rob.l", label_inv.cpu().numpy())
label_inv_array = np.load('rob.l.npy')
label_inv_array.shape

(50000,)

In [ ]:
loaded_array = np.load('rob.npy')
label_inv_array = np.load('rob.l.npy')

In [ ]:
loaded_array=np.load('rob.l.npy')